In [211]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

#RAIN_TYPE = 'Wheat'
#GRAIN_TYPE = 'newWheatData'
GRAIN_TYPE = 'WheatAdded_Type'
#GRAIN_TYPE = 'Combined_Grains'
# GRAIN_TYPE = 'Oats'
# GRAIN_TYPE = 'Barley'
# GRAIN_TYPE = 'Sorghum'
# GRAIN_TYPE = 'Soybeans'
# GRAIN_TYPE = 'Corn'
# Define lists to store the results for each fold
r2_scores_mc = []
mse_scores_mc = []
mae_scores_mc = []
min_abs_errors_mc = []
max_abs_errors_mc = []

r2_scores_density = []
mse_scores_density = []
mae_scores_density = []
min_abs_errors_density = []
max_abs_errors_density = []

max_error_indices_mc = []
max_error_indices_density = []


Load Dataset and handle processing

In [212]:
URL = "../../Datasets/processed/" + GRAIN_TYPE + ".csv"

# Load the dataset
df = pd.read_csv(URL)

# Encode categorical variables if 'Variety' is categorical
if df['Variety'].dtype == 'object':
    le = LabelEncoder()
    df['Variety'] = le.fit_transform(df['Variety'])

# Define the features and the target variables
X = df[['Freq', 'd(cm)', 'Attn', 'Phase_Corr', 'Permittivity_real', 'Permittivity_imaginary', 'Variety']]
y = df[['M%', 'Density']]

Now perform K-fold splitting and eval

In [213]:
# Initialize the KFold parameters
kf = KFold(n_splits=10, random_state=42, shuffle=True)

# Initialize the model
regressor = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_leaf=3)

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    regressor.fit(X_train, y_train)
    
    # Make predictions
    y_pred = regressor.predict(X_test)

    # Calculate metrics for moisture content
    r2_scores_mc.append(r2_score(y_test['M%'], y_pred[:, 0]))
    mse_scores_mc.append(mean_squared_error(y_test['M%'], y_pred[:, 0]))
    mae_scores_mc.append(mean_absolute_error(y_test['M%'], y_pred[:, 0]))
    abs_errors_mc = np.abs(y_test['M%'].values - y_pred[:, 0])
    min_abs_errors_mc.append(np.min(abs_errors_mc))
    max_abs_errors_mc.append(np.max(abs_errors_mc))
    # Find the index of the maximum absolute error for moisture content
    max_error_index_mc = np.argmax(np.abs(y_test['M%'].values - y_pred[:, 0]))
    max_error_indices_mc.append(test_index[max_error_index_mc])

    # Calculate metrics for density
    r2_scores_density.append(r2_score(y_test['Density'], y_pred[:, 1]))
    mse_scores_density.append(mean_squared_error(y_test['Density'], y_pred[:, 1]))
    mae_scores_density.append(mean_absolute_error(y_test['Density'], y_pred[:, 1]))
    abs_errors_density = np.abs(y_test['Density'].values - y_pred[:, 1])
    min_abs_errors_density.append(np.min(abs_errors_density))
    max_abs_errors_density.append(np.max(abs_errors_density))
    # Find the index of the maximum absolute error for density
    max_error_index_density = np.argmax(np.abs(y_test['Density'].values - y_pred[:, 1]))
    max_error_indices_density.append(test_index[max_error_index_density])


Now we will display accuracy metrics

In [214]:
# Print the average metrics for moisture content
print("Moisture Content Metrics:")
print("R^2:", np.mean(r2_scores_mc))
print("Mean Squared Error:", np.mean(mse_scores_mc))
print("Mean Absolute Error:", np.mean(mae_scores_mc))
print("Min Absolute Error:", np.mean(min_abs_errors_mc))
print("Max Absolute Error:", np.mean(max_abs_errors_mc))
# Print the average metrics for density
print("Density Metrics:")
print("R^2:", np.mean(r2_scores_density))
print("Mean Squared Error:", np.mean(mse_scores_density))
print("Mean Absolute Error:", np.mean(mae_scores_density))
print("Min Absolute Error:", np.mean(min_abs_errors_density))
print("Max Absolute Error:", np.mean(max_abs_errors_density))

print('\n\n', max_abs_errors_mc)

Moisture Content Metrics:
R^2: 0.9906791745154561
Mean Squared Error: 0.10445771129050589
Mean Absolute Error: 0.1540288729113959
Min Absolute Error: 3.197442310920451e-15
Max Absolute Error: 1.7602494349798659
Density Metrics:
R^2: 0.907767209653187
Mean Squared Error: 0.0003708076126859697
Mean Absolute Error: 0.013799200248918777
Min Absolute Error: 5.1376812733272545e-05
Max Absolute Error: 0.0687870627577299


 [0.7366986328039911, 1.827780528282947, 1.7191473307635423, 1.9295896820744805, 0.8459225770706151, 2.0551389081340403, 1.9439167534121378, 2.1771102497841905, 1.7662688530426234, 2.60092083443009]


In [215]:
import pandas as pd

# ... [rest of your code to perform the cross-validation]

# Function to print the details of rows with highest error in a formatted way
def print_high_error_rows(indices, data_frame, title):
    print(f"\n{title}")
    max_variety_length = max([len(str(data_frame.iloc[index]['Variety'])) for index in indices])
    # Print the header
    header = f"{'Index':<10}{'Variety':<{max_variety_length + 5}}{'Freq':<15}{'d(cm)':<15}{'M%':<15}{'Density':<15}{'Attn':<15}{'Phase':<15}{'Phase_Corr':<20}{'Permittivity_real':<20}{'Permittivity_imaginary':<20}"
    print(header)
    print("-" * len(header))
    # Print each row
    for index in indices:
        row = data_frame.iloc[index]
        formatted_row = f"{index:<10}{row['Variety']:<{max_variety_length + 5}}{row['Freq']:<15}{row['d(cm)']:<15}{row['M%']:<15}{row['Density']:<15}{row['Attn']:<15}{row['Phase']:<15}{row['Phase_Corr']:<20}{row['Permittivity_real']:<20}{row['Permittivity_imaginary']:<20}"
        print(formatted_row)

# Assuming max_error_indices_mc and max_error_indices_density are lists of indices for the highest errors
print_high_error_rows(max_error_indices_mc, df, "Rows with highest error for moisture content:")
print_high_error_rows(max_error_indices_density, df, "Rows with highest error for density:")



Rows with highest error for moisture content:
Index     Variety Freq           d(cm)          M%             Density        Attn           Phase          Phase_Corr          Permittivity_real   Permittivity_imaginary
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1128      2.0     6.0            7.7            14.6955519636445280.888347069728553312.038674473381452-86.12861959981063-419.9021090578576  3.128073492647212   0.4931984205905987  
1110      3.0     16.0           7.7            11.2592006402320820.894633248600148320.394764212490657130.50463887631838-969.7236225847512  2.705087806648223   0.3122746184985375  
1053      1.0     8.0            7.7            14.4513410694344180.827404736061059814.765396504270877-162.89355451474927-539.6713635377673  2.9338101277525843  0.4765752718018713  
902       0.0     12.0           7.7            16.0994632439387250